In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

## Predicting the Genre of Books from Summaries

We'll use a set of book summaries from the [CMU Book Summaries Corpus](http://www.cs.cmu.edu/~dbamman/booksummaries.html) in this experiment.  This contains a large number of summaries (16,559) and includes meta-data about the genre of the books taken from Freebase.  Each book can have more than one genre and there are 227 genres listed in total.  To simplify the problem of genre prediction we will select a small number of target genres that occur frequently in the collection and select the books with these genre labels.  This will give us one genre label per book. 

Your goal in this portfolio is to take this data and build a predictive model to classify the books into one of the five target genres.  You will need to extract suitable features from the texts and select a suitable model to classify them. You should build at least one model but you could build two and compare the results if you have time.

You should report on each stage of your experiment as you work with the data.


## Data Preparation

The first task is to read the data. It is made available in tab-separated format but has no column headings. We can use `read_csv` to read this but we need to set the separator to `\t` (tab) and supply the column names.  The names come from the [ReadMe](data/booksummaries/README.txt) file.

In [2]:
names = ['wid', 'fid', 'title', 'author', 'date', 'genres', 'summary']

books = pd.read_csv("data/booksummaries/booksummaries.txt", sep="\t", header=None, names=names, keep_default_na=False)
books.head()

,wid,fid,title,author,date,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,,,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


We next filter the data so that only our target genre labels are included and we assign each text to just one of the genre labels.  It's possible that one text could be labelled with two of these labels (eg. Science Fiction and Fantasy) but we will just assign one of those here. 

In [3]:
target_genres = ["Children's literature",
                 'Science Fiction',
                 'Novel',
                 'Fantasy',
                 'Mystery']

# create a Series of empty strings the same length as the list of books
genre = pd.Series(np.repeat("", books.shape[0]))
# look for each target genre and set the corresponding entries in the genre series to the genre label
for g in target_genres:
    genre[books['genres'].str.contains(g)] = g

# add this to the book dataframe and then select only those rows that have a genre label
# drop some useless columns
books['genre'] = genre
genre_books = books[genre!=''].drop(['genres', 'fid', 'wid'], axis=1)

genre_books.shape

(8954, 5)

In [4]:
# check how many books we have in each genre category
genre_books.groupby('genre').count()

,title,author,date,summary
genre,,,,
Children's literature,1092,1092,1092,1092
Fantasy,2311,2311,2311,2311
Mystery,1396,1396,1396,1396
Novel,2258,2258,2258,2258
Science Fiction,1897,1897,1897,1897


## Modelling

Now you take over to build a suitable model and present your results

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import f1_score, accuracy_score

In [6]:
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

In [7]:
genre_books['clean_summary'] = genre_books['summary'].apply(lambda x: clean_summary(x))
genre_books.head(1)

,title,author,date,summary,genre,clean_summary
0,Animal Farm,George Orwell,1945-08-17,"Old Major, the old boar on the Manor Farm, ca...",Children's literature,old major the old boar on the manor farm calls...


In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

genre_books['clean_summary'] = genre_books['clean_summary'].apply(lambda x: remove_stopwords(x))

In [9]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(genre_books['genre'])

y = multilabel_binarizer.transform(genre_books['genre'])

In [10]:
x_train, x_val, ytrain, yval = train_test_split(genre_books['clean_summary'],
                                              y, test_size=0.2)

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
xtrain = tfidf_vectorizer.fit_transform(x_train)
xval = tfidf_vectorizer.transform(x_val)

In [12]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)
clf.fit(xtrain, ytrain)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [13]:
y_pred = clf.predict(xval)
f1_score(yval, y_pred, average="micro"), accuracy_score(yval, y_pred)

(0.7438793936085331, 0.3975432719151312)

In [14]:
pred_prob = clf.predict_proba(xval)

In [15]:
t = 0.3
predp = (pred_prob >= t).astype(int)
f1_score(yval, predp, average="micro"), accuracy_score(yval, predp)

(0.7602610684968376, 0.2283640424343942)

In [16]:
def predict(m):
    m = clean_summary(m)
    m = remove_stopwords(m)
    m_vec = tfidf_vectorizer.transform([m])
    m_pred = clf.predict(m_vec)
    return multilabel_binarizer.inverse_transform(m_pred)
for i in range(10):
    k = x_val.sample(1).index[0]
    print("Book: ", genre_books['title'][k], 
          "\nPredicted genre: ", predict(x_val[k])) ,
    print("Actual genre: ",genre_books['genre'][k], "\n")

Book:  Amber and Iron 
Predicted genre:  [('F', 'a', 'n', 's', 't', 'y')]
Actual genre:  Fantasy 

Book:  The Enemy 
Predicted genre:  [('e', 'r', 's', 't', 'y')]
Actual genre:  Mystery 

Book:  Search For Senna 
Predicted genre:  [('e', 'l', 's', 't')]
Actual genre:  Novel 

Book:  The Cat Who Saw Red 
Predicted genre:  [('e', 'l', 'r', 's', 't')]
Actual genre:  Mystery 

Book:  A Dog's Life: The Autobiography of a Stray 
Predicted genre:  [('a', 'e', 'n', 's', 't')]
Actual genre:  Children's literature 

Book:  Flash 
Predicted genre:  [('e', 'o', 't')]
Actual genre:  Science Fiction 

Book:  Dragonsdale 
Predicted genre:  [('F', 'a', 'n', 's', 't', 'y')]
Actual genre:  Fantasy 

Book:  The Bishop's Heir 
Predicted genre:  [('F', 'a', 'n', 's', 't', 'y')]
Actual genre:  Fantasy 

Book:  Teranesia 
Predicted genre:  [('N', 'e', 'l', 'o', 'v')]
Actual genre:  Science Fiction 

Book:  The Road to Wellville 
Predicted genre:  [('e', 'l', 'o', 't')]
Actual genre:  Novel 

